In [3]:
import os
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Cargar el archivo .env
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)



True

In [4]:
# Configuración de PostgreSQL igual que en docker-compose
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
database = os.getenv('DB_NAME')
schema = "public_raw"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Crear el schema si no existe
with engine.connect() as conn:
    conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema}"))
    conn.commit()

# Carpeta con archivos parquet
parquet_folder = Path('../data')

for file in parquet_folder.iterdir():
    # Leer parquet
    df = pd.read_parquet(file)

    # Guardar en PostgreSQL
    table_name = "raw_" + file.stem  # nombre de la tabla igual al archivo
    df.to_sql(table_name, engine, schema=schema, if_exists='replace', index=False)
    print(f"Tabla '{schema}.{table_name}' cargada correctamente")


Tabla 'public_raw.raw_products' cargada correctamente
Tabla 'public_raw.raw_users' cargada correctamente
Tabla 'public_raw.raw_orders' cargada correctamente
Tabla 'public_raw.raw_reviews' cargada correctamente
